In [4]:
import torch
from torch import nn
from torch.autograd import Variable
import pandas as pd
import dateutil
import datetime
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as funcs

In [5]:
# 68,335,729 data points in total
import random
p = 0.0001  # 0.01% of the lines -> 6800 data points
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
"""
df = pd.read_csv(
         'data/all_events.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
"""
# df = pd.read_csv('../data/Kiasma_sorted.csv', nrows=500000, low_memory=False)
# df = pd.read_csv('../data/Rautatientoriitä_sorted.csv', low_memory=False)
# df = pd.read_csv('../data/AaltoyliopistoMKorkeakouluaukio_sorted.csv', low_memory=False)
# df = pd.read_csv('../data/Arabiankauppakeskus_sorted.csv', low_memory=False)
df = pd.read_csv('../data/all_events.csv', nrows=2000000, low_memory=False)

## Drop duplicated based on timestamp

In [6]:
df = df.drop_duplicates('ts').reset_index()

In [7]:
df.shape

(42058, 9)

## Add some useful columns

In [8]:
# sort based on timestamp
df = df.sort_values('ts')
# the hour and minutes of the timestamp
df['hour_and_minutes'] = df['ts'].apply(lambda x: x[11:16])
# month of timestamp
df['month'] = df['ts'].apply(lambda x: x[5:7])
# the percentage of bikes
df['bikes_percent'] = df['bikes'] / df['total_slots']
df['minutes_from_midnight'] = df['ts'].apply(lambda x: int(x[11:13]) * 60 + int(x[14:16]))
df['hour'] = df['hour_and_minutes'].apply(lambda x: x[:2])
df['date'] = df['ts'].apply(pd.to_datetime)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['weekend'] = df['weekday'].apply(lambda x: x > 4)
df.head()

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend
34966,1654489,2016-04-26T14:13:26,60.173099,24.949636,1,A21,Varsapuistikko,0,28,14:13,04,0.0,853,14,2016-04-26 14:13:26,1,False
34994,1654931,2016-04-26T17:27:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:27,04,0.0,1047,17,2016-04-26 17:27:01,1,False
34993,1654928,2016-04-26T17:28:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:28,04,0.0,1048,17,2016-04-26 17:28:01,1,False
35016,1655257,2016-04-26T17:29:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:29,04,0.0,1049,17,2016-04-26 17:29:01,1,False
34960,1654420,2016-04-26T17:30:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:30,04,0.0,1050,17,2016-04-26 17:30:01,1,False


## Prepare for predictions

### Add column which states whether a bike was added at a given moment

In [9]:
df['bike_added'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] < x['bikes'] , axis=1)
df['bike_removed'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] > x['bikes'] , axis=1)
df.head(100)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed
34966,1654489,2016-04-26T14:13:26,60.173099,24.949636,1,A21,Varsapuistikko,0,28,14:13,04,0.000000,853,14,2016-04-26 14:13:26,1,False,False,True
34994,1654931,2016-04-26T17:27:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:27,04,0.000000,1047,17,2016-04-26 17:27:01,1,False,False,True
34993,1654928,2016-04-26T17:28:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:28,04,0.000000,1048,17,2016-04-26 17:28:01,1,False,False,True
35016,1655257,2016-04-26T17:29:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:29,04,0.000000,1049,17,2016-04-26 17:29:01,1,False,False,True
34960,1654420,2016-04-26T17:30:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:30,04,0.000000,1050,17,2016-04-26 17:30:01,1,False,False,True
34982,1654758,2016-04-26T17:41:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:41,04,0.000000,1061,17,2016-04-26 17:41:01,1,False,False,True
34961,1654423,2016-04-26T17:45:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:45,04,0.000000,1065,17,2016-04-26 17:45:01,1,False,False,True
34988,1654862,2016-04-26T17:47:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:47,04,0.000000,1067,17,2016-04-26 17:47:01,1,False,False,True
35019,1655296,2016-04-26T17:48:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:48,04,0.000000,1068,17,2016-04-26 17:48:01,1,False,False,True
35010,1655152,2016-04-27T07:21:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,07:21,04,0.000000,441,07,2016-04-27 07:21:01,2,False,False,True


### Add max waiting time for next bike

In [58]:
# extract difference in minutes to a separate column
# on the moment that a bike was added, the waiting time is the time to next bike
i = 0
first_non_additive_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_added'] == False):
        i += 1
    if (i >= len(df)):
        break
    additive_row = df.iloc[i]
    time = additive_row['minutes_from_midnight']
    difference = abs(time - df.iloc[first_non_additive_index]['minutes_from_midnight'])
    df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike'] = difference 
    # add time using a date object (more accurate if wait for more than 24h)
    date_time = additive_row['date']
    last_date = df.iloc[first_non_additive_index]['date']
    date_difference = abs((date_time - last_date).total_seconds() / 60)
    df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike_as_date'] = date_difference 
    first_non_additive_index = i
    i += 1
# if we can't know when the next bike will be added, fill with -1 for now
df = df.fillna(-1)
df.head(100)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,...,weekday,weekend,bike_added,bike_removed,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for,wait_for_bike_taken_as_date,wait_for_bike_taken,max_wait_for_new_bike
34966,1654489,2016-04-26T14:13:26,60.173099,24.949636,1,A21,Varsapuistikko,0,28,14:13,...,1,False,False,True,-1.000000,133.000000,-1.0,-1.0,-1.0,-1.0
34994,1654931,2016-04-26T17:27:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:27,...,1,False,False,True,1.000000,190.000000,-1.0,-1.0,-1.0,1.0
34993,1654928,2016-04-26T17:28:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:28,...,1,False,False,True,5.000000,185.000000,-1.0,-1.0,-1.0,5.0
35016,1655257,2016-04-26T17:29:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:29,...,1,False,False,True,1.000000,58.000000,-1.0,-1.0,-1.0,1.0
34960,1654420,2016-04-26T17:30:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:30,...,1,False,False,True,1.000000,126.000000,-1.0,-1.0,-1.0,1.0
34982,1654758,2016-04-26T17:41:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:41,...,1,False,False,True,-1.000000,161.000000,-1.0,-1.0,-1.0,-1.0
34961,1654423,2016-04-26T17:45:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:45,...,1,False,False,True,2.000000,127.000000,-1.0,-1.0,-1.0,2.0
34988,1654862,2016-04-26T17:47:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:47,...,1,False,False,True,1.000000,177.000000,-1.0,-1.0,-1.0,1.0
35019,1655296,2016-04-26T17:48:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:48,...,1,False,False,True,-1.000000,65.000000,-1.0,-1.0,-1.0,-1.0
35010,1655152,2016-04-27T07:21:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,07:21,...,2,False,False,True,3.000000,39.000000,-1.0,-1.0,-1.0,3.0


### Add idle time for each station

In [11]:
i = 0
idle_for = 0
idle_for_as_date = 0
while (i < len(df)):
    row = df.iloc[i]
    previous_row = df.iloc[(i-1 if i > 0 else 0)]
    if (not row['bikes'] == previous_row['bikes']):
        idle_for = 0
        idle_for_as_date = 0
        df.loc[i, 'idle_for'] = idle_for
        df.loc[i, 'idle_for_as_date'] = idle_for_as_date        
        i+= 1
        continue
    time_now = row['minutes_from_midnight']
    time_last = previous_row['minutes_from_midnight']
    idle_for += abs(time_now - time_last)
    df.loc[i, 'idle_for'] = idle_for
    # add time using a date object (more accurate if wait for more than 24h)
    date_time_now = row['date']
    date_time_last = previous_row['date']
    secs = (date_time_now - date_time_last).total_seconds()
    idle_for_as_date += abs(secs / 60)
    df.loc[i, 'idle_for_as_date'] = idle_for_as_date
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,...,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for
34966,1654489,2016-04-26T14:13:26,60.173099,24.949636,1,A21,Varsapuistikko,0,28,14:13,...,853,14,2016-04-26 14:13:26,1,False,False,True,-1.000000,133.000000,-1.0
34994,1654931,2016-04-26T17:27:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:27,...,1047,17,2016-04-26 17:27:01,1,False,False,True,1.000000,190.000000,-1.0
34993,1654928,2016-04-26T17:28:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:28,...,1048,17,2016-04-26 17:28:01,1,False,False,True,5.000000,185.000000,-1.0
35016,1655257,2016-04-26T17:29:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:29,...,1049,17,2016-04-26 17:29:01,1,False,False,True,1.000000,58.000000,-1.0
34960,1654420,2016-04-26T17:30:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:30,...,1050,17,2016-04-26 17:30:01,1,False,False,True,1.000000,126.000000,-1.0
34982,1654758,2016-04-26T17:41:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:41,...,1061,17,2016-04-26 17:41:01,1,False,False,True,-1.000000,161.000000,-1.0
34961,1654423,2016-04-26T17:45:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:45,...,1065,17,2016-04-26 17:45:01,1,False,False,True,2.000000,127.000000,-1.0
34988,1654862,2016-04-26T17:47:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:47,...,1067,17,2016-04-26 17:47:01,1,False,False,True,1.000000,177.000000,-1.0
35019,1655296,2016-04-26T17:48:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,17:48,...,1068,17,2016-04-26 17:48:01,1,False,False,True,-1.000000,65.000000,-1.0
35010,1655152,2016-04-27T07:21:01,60.173099,24.949636,1,A21,Varsapuistikko,0,28,07:21,...,441,07,2016-04-27 07:21:01,2,False,False,True,3.000000,39.000000,-1.0


In [12]:
df.describe()

,index,lat,lon,operative,bikes,total_slots,bikes_percent,minutes_from_midnight,weekday,max_wait_for_new_bike_as_date,idle_for_as_date,idle_for
count,4.205800e+04,42058.000000,42058.000000,42058.0,42058.000000,42058.000000,42058.000000,42058.000000,42058.000000,42058.000000,42058.000000,42058.000000
mean,9.957349e+05,60.155415,24.950391,1.0,16.460031,29.999524,0.548668,751.202601,3.120572,9.466110,112.374576,-0.898046
std,5.756032e+05,0.000273,0.000012,0.0,5.845913,0.030836,0.194864,355.569997,1.981940,428.572756,1888.191510,0.302592
min,0.000000e+00,60.155411,24.949636,1.0,0.000000,28.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000
25%,4.974928e+05,60.155411,24.950391,1.0,12.000000,30.000000,0.400000,467.000000,1.000000,2.000000,3.000000,-1.000000
50%,9.950945e+05,60.155411,24.950391,1.0,17.000000,30.000000,0.566667,755.000000,3.000000,2.000000,14.000000,-1.000000
75%,1.492717e+06,60.155411,24.950391,1.0,20.000000,30.000000,0.666667,1039.000000,5.000000,3.000000,55.000000,-1.000000
max,1.999985e+06,60.173099,24.950391,1.0,45.000000,30.000000,1.500000,1439.000000,6.000000,87848.983333,88076.000000,0.000000


### Add time for next bike take

Predict how much time for the next bike to be taken: max waiting time is not useful here.

In [42]:
i = 0
first_non_removing_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_removed'] == False):
        i += 1
    if (i >= len(df)):
        break
    removing_row = df.iloc[i]
#     time = removing_row['minutes_from_midnight']
#     times_until_now = df.loc[first_non_removing_index:i-1, 'minutes_from_midnight']
#     difference = abs(time - times_until_now)
#     df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'wait_for_bike_taken'] = difference 
    # add time using a date object (more accurate if wait for more than 24h)
    date_time = removing_row['date']
    date_times_until_now = df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'date']
    date_difference = abs((date_time - date_times_until_now).apply(lambda x: x.total_seconds() / 60))
    try:
        df.loc[first_non_removing_index:i-1, 'wait_for_bike_taken_as_date'] = date_difference 
    except:
        asd = date_difference
        print(date_times_until_now)
        print(len(df.loc[first_non_removing_index:(i if i > 0 else 0)])) 
        print(first_non_removing_index)
        print('--')
    first_non_removing_index = i
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

ValueError: Must have equal len keys and value when setting with an iterable

In [43]:
max_wait_counts = df['max_wait_for_new_bike_as_date'].value_counts()
wait_counts = df['wait_for_bike_taken_as_date'].value_counts()
print(max_wait_counts)
print(wait_counts)

 2.000000        30029
 3.000000         4465
 56.000000        2979
 37.000000         787
 15.000000         632
 7.000000          578
 6.000000          575
 16.000000         417
 13.000000         394
 12.000000         325
 4.000000          174
 11.000000         128
 8.000000          125
 27.000000         119
 1.000000          104
 9.000000           92
 1.983333           39
 19.016667          27
-1.000000           26
 4.983333            7
 19.000000           7
 5.000000            6
 1.016667            4
 10.000000           4
 20.000000           4
 0.983333            4
 87848.983333        1
 4.016667            1
 17.983333           1
 11.983333           1
 25.000000           1
 2.983333            1
 9.016667            1
Name: max_wait_for_new_bike_as_date, dtype: int64
37889.000000    2
40976.000000    2
1034.000000     2
37906.000000    2
1206.000000     2
37920.000000    2
6355.000000     2
37807.000000    2
1207.000000     2
40971.000000    2
37963.00000

## Remove useless or codependent columns

In [62]:
store_df = df
cleaned_df = df.drop(columns=['index', 'ts', 'sid', 'hour_and_minutes', 'name', 'bike_added', 'bike_removed', 'lat', 'lon', 'bikes', 'total_slots', 'date'])
filterable_max_waits = max_wait_counts[max_wait_counts < 10].index
filterable_waits = wait_counts[wait_counts < 10].index
cleaned_df = cleaned_df[~cleaned_df['max_wait_for_new_bike'].isin(filterable_max_waits)]
# cleaned_df = cleaned_df[~cleaned_df['wait_for_bike_taken_as_date'].isin(filterable_waits)]
print('removed', len(df) + len(cleaned_df), 'rows')
print('using columns', cleaned_df.columns.values)

removed 84060 rows
using columns ['operative' 'month' 'bikes_percent' 'minutes_from_midnight' 'hour'
 'weekday' 'weekend' 'max_wait_for_new_bike_as_date' 'idle_for_as_date'
 'idle_for' 'wait_for_bike_taken_as_date' 'wait_for_bike_taken'
 'max_wait_for_new_bike']


## Add classes for prediciton

In [63]:
def define_class(x):
    if x < 5:
        return 0
    if x < 15:
        return 1
    return 2

cleaned_df['max_wait_for_new_bike_class'] = cleaned_df['max_wait_for_new_bike'].apply(define_class)
# cleaned_df['wait_for_bike_taken_class'] = cleaned_df['wait_for_bike_taken_as_date'].apply(define_class)
# print('value count of classes in wait for next bike taken:')
# print(cleaned_df['wait_for_bike_taken_class'].value_counts())
print()
print('value count of classes in max wait for new bike')
print(cleaned_df['max_wait_for_new_bike_class'].value_counts())


value count of classes in max wait for new bike
0    34847
1     7154
2        1
Name: max_wait_for_new_bike_class, dtype: int64


## Define training and test sets

In [64]:
# drop the columns we would not know in a real situation, as well as the column to be predicted
X = cleaned_df.drop(columns=['max_wait_for_new_bike_class', 'max_wait_for_new_bike_as_date', 'wait_for_bike_taken_as_date']).as_matrix()
y = cleaned_df['max_wait_for_new_bike_class'].as_matrix()
# set 
train_size = int(len(X)*0.8)

# define the training and test sets
X_train = torch.from_numpy(np.asarray(X[:train_size].astype('float')))
# take transpose to get right dimensions for torch
y_train = torch.from_numpy(np.asarray([y[:train_size]]).astype('float').T)
X_test = torch.from_numpy(np.asarray(X[train_size:].astype('float')))
y_test = torch.from_numpy(np.asarray([y[train_size:]]).astype('float').T)
# combine input and labels
train_data = torch.cat((X_train, y_train), 1)
test_data = torch.cat((X_test, y_test), 1)

# sanity check
assert len(X) == len(X_train) + len(X_test)

## Define a simple neural network

In [65]:
class Net(nn.Module):
    def __init__(self, in_neurons=11, out_neurons=3, nr_layers=1, nr_neurons=10):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(nr_layers):
            self.layers.append(nn.Linear(in_neurons, nr_neurons))
            in_neurons = nr_neurons
        self.output_layer = nn.Linear(in_neurons, out_neurons)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = funcs.relu(x)
        return self.output_layer(x)

## Define loss function

In [52]:
loss_func = torch.nn.CrossEntropyLoss()

## Define training loop

In [66]:
net = Net().double()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
data_len = len(train_data)
batch_size = 250
epochs = 50
for epoch in range(epochs):
    # shuffle the data
    indices = torch.randperm(data_len)
    # train the data in batches
    for i in range(0, data_len, batch_size):
        batch_indices = indices[i: i+ batch_size]
        data = train_data[batch_indices]
        # assume that last cell in array is label
        inp, label = data[:, :data.shape[1]-1], data[:, data.shape[1]-1]
        # torch has to be fed variables
        input_var = Variable(inp)
        label_var = Variable(label.long())
        # reset parameters after each training round
        optimizer.zero_grad()
        # actual forward pass
        outputs = net(input_var)
        # calculate loss
        loss = loss_func(outputs, label_var)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print('e:', epoch)
        print('loss', loss.item())
        print('--')

print('training done with a training set of length', len(train_data))

e: 0
loss 0.011217853767097214
--
e: 10
loss 6.320257158112769e-05
--
e: 20
loss 2.9847046717897215e-06
--
e: 30
loss 0.04352020912907271
--
e: 40
loss 0.006553049503505284
--
training done with a training set of length 33601


## Evaluate with test set

In [69]:
total_loss = 0
inp, label = test_data[:, :test_data.shape[1]-1], test_data[:, test_data.shape[1]-1]
test_input_var = Variable(inp)
test_label_var = Variable(label.long())
test_outputs = net(test_input_var)
# just print out the loss for now
test_loss = loss_func(test_outputs, test_label_var)
# this is basically just to see whether the net always predicted the same thing
total_loss = test_loss.item()
probs = funcs.softmax(test_outputs, dim=1)
predictions = torch.round(funcs.softmax(test_outputs, dim=1))
i = 0
for true, pred in zip(test_label_var.data, funcs.softmax(test_outputs, dim=1).data):
    if i == 200:
        break
    print(true, pred)
    i += 1

tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.floa